<a href="https://colab.research.google.com/github/3Techies/Bias-Detection-ML-models/blob/main/notebooks/02_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!git clone https://github.com/3Techies/Bias-Detection-ML-models.git


Cloning into 'Bias-Detection-ML-models'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 162 (delta 17), reused 6 (delta 3), pack-reused 131 (from 1)
Receiving objects: 100% (162/162), 792.22 KiB | 13.66 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [61]:
%cd Bias-Detection-ML-models/notebooks

/content/Bias-Detection-ML-models/Bias-Detection-ML-models/Bias-Detection-ML-models/notebooks


In [62]:
import sys
import os

print("CWD:", os.getcwd())

sys.path.append(os.path.abspath(".."))

print("Files in root:", os.listdir(".."))

import config
print("SUCCESS: config imported")

CWD: /content/Bias-Detection-ML-models/Bias-Detection-ML-models/Bias-Detection-ML-models/notebooks
Files in root: ['README.md', 'how-to-use-git.md', 'config.py', 'notebooks', 'results', 'LICENSE', 'data', '.git', '.gitignore']
SUCCESS: config imported


In [63]:
import pandas as pd
%cd ..
print(config.DATA_PATH)

/content/Bias-Detection-ML-models/Bias-Detection-ML-models/Bias-Detection-ML-models
data/loan_data.csv


In [64]:
df = pd.read_csv(config.DATA_PATH)
df.head(10)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
5,21.0,female,High School,12951.0,0,OWN,2500.0,VENTURE,7.14,0.19,2.0,532,No,1
6,26.0,female,Bachelor,93471.0,1,RENT,35000.0,EDUCATION,12.42,0.37,3.0,701,No,1
7,24.0,female,High School,95550.0,5,RENT,35000.0,MEDICAL,11.11,0.37,4.0,585,No,1
8,24.0,female,Associate,100684.0,3,RENT,35000.0,PERSONAL,8.90,0.35,2.0,544,No,1
9,21.0,female,High School,12739.0,0,OWN,1600.0,VENTURE,14.74,0.13,3.0,640,No,1


In [85]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [66]:
target_column="loan_status"  #selecting the target_column.
X = df.drop(columns=[target_column]) #isolate the features from dataset,by droping the target feature.
y = df[target_column] #giving y the df of target column

In [67]:
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()#The categorical columns from the input features.
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist() #The numerical columns from the input features.

print("categorical columns: ",categorical_cols)
print("numerical columns: ",numerical_cols)


categorical columns:  ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']
numerical columns:  ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']


In [68]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)


In [69]:
X_train, X_test, y_train, y_test = train_test_split( #defining the training dataset and the testing dataset.
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [70]:
dt_model = DecisionTreeClassifier(  #describing the model with our requirements.
    criterion="gini",
    max_depth=6,
    min_samples_leaf=50,
    random_state=42
)

In [71]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    criterion="gini",
    max_depth=6)
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42)
gr_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6
)

In [72]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", dt_model)
    ]
)

In [73]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score'])])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, min_samples_leaf=50,
                                        random_state=42))])

In [74]:
y_pred_dt = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_dt[:5])

[0 0 0 0 1]


In [75]:
print("Accuracy:", accuracy_score(y_test, y_pred_dt)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

Accuracy: 91.2 %

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      7000
           1       0.86      0.72      0.78      2000

    accuracy                           0.91      9000
   macro avg       0.89      0.84      0.86      9000
weighted avg       0.91      0.91      0.91      9000



In [76]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", rf_model)
    ]
)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_cred_hist_length',
                                                   'credit_score'])])),
                ('model', RandomForestClassifier(max_depth=6))])

In [77]:
y_pred_rf = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_rf[:5])
print("Accuracy:", accuracy_score(y_test, y_pred_rf)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

[0 0 0 0 1]
Accuracy: 91.98888888888888 %

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      7000
           1       0.92      0.70      0.79      2000

    accuracy                           0.92      9000
   macro avg       0.92      0.84      0.87      9000
weighted avg       0.92      0.92      0.92      9000



In [78]:
pipeline=Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", xgb_model)
    ]
)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['person_gender',
                                                   'person_education',
                                                   'person_home_ownership',
                                                   'loan_intent',
                                                   'previous_loan_defaults_on_file']),
                                                 ('num', 'passthrough',
                                                  ['person_age',
                                                   'person_income',
                                                   'person_emp_exp',
                                                   'loan_amnt', 'loan_int_rate',
                                                   'loan_percent_income',
                                                   'cb_person_c...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [79]:
y_pred_xgb = pipeline.predict(X_test) #Finally testing the dT model by providing it with testing data.
print(y_pred_xgb[:5])
print("Accuracy:", accuracy_score(y_test, y_pred_xgb)*100,"%") #calculating the accuracy of the model.
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))

[0 0 0 0 1]
Accuracy: 93.14444444444445 %

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.96      7000
           1       0.90      0.78      0.83      2000

    accuracy                           0.93      9000
   macro avg       0.92      0.88      0.90      9000
weighted avg       0.93      0.93      0.93      9000



In [80]:
# Create pipeline with Logistic Regression
model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(max_iter=2000)) # Increased max_iter
    ]
)

In [81]:
# Train the model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Model Accuracy:", accuracy)


Model Accuracy: 0.8876666666666667


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [82]:
mod_names = ["Decision Tree", "Random Forest", "XGBoost", "Logistic Regression"]
mod_results = [y_pred_dt, y_pred_rf, y_pred_xgb, y_pred]

In [87]:
for i, mod_name in enumerate(mod_names):
    print(f"Model: {mod_name}")
    priv_tot = 0
    priv_appr = 0
    unpriv_tot = 0
    unpriv_appr = 0
    res = mod_results[i]
    for i in range(len(X_test)):
        if(X_test.iloc[i]['person_gender'].lower() == config.PRIVILEGED_GROUP.lower()):
          priv_tot += 1
          if(res[i]==1):
            priv_appr += 1
        else:
          unpriv_tot += 1
          if(res[i]==1):
            unpriv_appr += 1
    print(f"{config.PRIVILEGED_GROUP} -> {priv_tot} samples, {priv_appr} approved")
    print("approval rate of privileged group:", priv_appr/priv_tot)
    print(f"{config.UNPRIVILEGED_GROUP} -> {unpriv_tot} samples, {unpriv_appr} approved")
    print("approval rate of unprivileged group:", unpriv_appr/unpriv_tot)
    print(f"dp = {(priv_appr/priv_tot - unpriv_appr/unpriv_tot)*100}")
    print()


Model: Decision Tree
Male -> 5055 samples, 955 approved
approval rate of privileged group: 0.18892185954500496
Female -> 3945 samples, 705 approved
approval rate of unprivileged group: 0.17870722433460076
dp = 1.0214635210404195

Model: Random Forest
Male -> 5055 samples, 858 approved
approval rate of privileged group: 0.16973293768545994
Female -> 3945 samples, 659 approved
approval rate of unprivileged group: 0.16704689480354878
dp = 0.2686042881911155

Model: XGBoost
Male -> 5055 samples, 987 approved
approval rate of privileged group: 0.19525222551928784
Female -> 3945 samples, 752 approved
approval rate of unprivileged group: 0.19062103929024082
dp = 0.4631186229047013

Model: Logistic Regression
Male -> 5055 samples, 1113 approved
approval rate of privileged group: 0.2201780415430267
Female -> 3945 samples, 836 approved
approval rate of unprivileged group: 0.21191381495564005
dp = 0.8264226587386653



In [96]:
# sensitive attribute column from original dataframe
sensitive_test = X_test['person_gender']
def confusion_by_group(y_true, y_pred, sensitive_series, group_value):
    idx = sensitive_series == group_value
    y_true_g = y_true[idx]
    y_pred_g = y_pred[idx]
    tn, fp, fn, tp = confusion_matrix(y_true_g, y_pred_g).ravel()
    return tn, fp, fn, tp
male = config.PRIVILEGED_GROUP.lower()
female = config.UNPRIVILEGED_GROUP.lower()
def rates(tn, fp, fn, tp):
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    return fnr, tpr

In [98]:
for i, mod_name in enumerate(mod_names):
    print(f"Model: {mod_name}")
    res = mod_results[i]
    cm = confusion_matrix(y_test, res)
    tn_m, fp_m, fn_m, tp_m = confusion_by_group( y_test, res, sensitive_test, male)
    tn_f, fp_f, fn_f, tp_f = confusion_by_group( y_test, res, sensitive_test, female )
    print("Confusion Matrix for ",male)
    print(f"""
    Confusion Matrix
    ----------------
    TP: {tp_m} | FP: {fp_m}
    FN: {fn_m} | TN: {tn_m}
    """)
    print()
    print("Confusion Matrix for ",female)
    print(f"""
    Confusion Matrix
    ----------------
    TP: {tp_f} | FP: {fp_f}
    FN: {fn_f} | TN: {tn_f}
    """)
    print()
    fnr_m, tpr_m = rates(tn_m, fp_m, fn_m, tp_m)
    fnr_f, tpr_f = rates(tn_f, fp_f, fn_f, tp_f)
    print(f"Male   → FNR: {fnr_m:.2f}, TPR: {tpr_m:.2f}, Female → FNR: {fnr_f:.2f}, TPR: {tpr_f:.2f}")
    print(f"dp = {(tpr_m - tpr_f)*100}")
    print()


Model: Decision Tree
Confusion Matrix for  male

    Confusion Matrix
    ----------------
    TP: 824 | FP: 131
    FN: 322 | TN: 3778
    

Confusion Matrix for  female

    Confusion Matrix
    ----------------
    TP: 610 | FP: 95
    FN: 244 | TN: 2996
    

Male   → FNR: 0.28, TPR: 0.72, Female → FNR: 0.29, TPR: 0.71
dp = 0.473697332336076

Model: Random Forest
Confusion Matrix for  male

    Confusion Matrix
    ----------------
    TP: 796 | FP: 62
    FN: 350 | TN: 3847
    

Confusion Matrix for  female

    Confusion Matrix
    ----------------
    TP: 602 | FP: 57
    FN: 252 | TN: 3034
    

Male   → FNR: 0.31, TPR: 0.69, Female → FNR: 0.30, TPR: 0.70
dp = -1.0328154950934088

Model: XGBoost
Confusion Matrix for  male

    Confusion Matrix
    ----------------
    TP: 892 | FP: 95
    FN: 254 | TN: 3814
    

Confusion Matrix for  female

    Confusion Matrix
    ----------------
    TP: 669 | FP: 83
    FN: 185 | TN: 3008
    

Male   → FNR: 0.22, TPR: 0.78, Female → FNR: